# Sounding Data from Wyoming

- Acquiring data with `siphon`
    - Cases (up to two days earlier):
        - 2016-12-25
        - 2017-01-31
        - 2017-03-14
        - 2017-11-15
        - 2017-11-16
  
  
- Processing data with `metpy`
    - Generating Skew-T Log-P maps
    
Based on [Skew-T with Complex Layout Example (MetPy documentation)](https://unidata.github.io/MetPy/latest/examples/plots/Skew-T_Layout.html#sphx-glr-examples-plots-skew-t-layout-py).

## Loading necessary packages

In [1]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
import pandas as pd

import datetime
from siphon.simplewebservice.wyoming import WyomingUpperAir

import metpy.calc as mpcalc
from metpy.cbook import get_test_data
from metpy.plots import add_metpy_logo, Hodograph, SkewT
from metpy.units import units

## Defining necessary functions

### `acquire_sounding_data`

Using the date of the case:

- List dates up to 96h before, from 12 to 12 hours
- Get sounding data for each date using `siphon`

In [2]:
def acquire_sounding_data(case):
    dates = [case - datetime.timedelta(hours=x) for x in range(0, 96, 12)]
    soundings = []

    for i in dates:
        try: soundings.append(WyomingUpperAir.request_data(i, station))
        except ValueError: pass
        
    return soundings

### `pressure_interval`

Using sounding data, define less barbs points for the plot

In [3]:
def pressure_interval(p, u, v, upper=100, lower=1000, spacing=50):

    intervals = list(range(upper,lower,spacing))

    ix = []
    for center in intervals:
        index = (np.abs(p-center)).argmin()
        if index not in ix:
            ix.append(index)

    return p[ix],u[ix],v[ix]

### `extract_vars_plot_save_figs`

Using sounding data, station index and filepath:

- Extract variables necessary for the plot
- Calculate LCL, full parcel profile, bigger interval for barbs (`pressure_interval`), CAPE, CIN
- Plot using `SkewT`
- Save in the filepath

In [4]:
def extract_vars_plot_save_figs(sounding, station, save_path):
    #-- Extracting variables
    sounding = sounding.dropna(subset=('pressure', 'temperature', 'dewpoint', 'u_wind', 'v_wind'), how='all')
    p = sounding['pressure'].values
    T = sounding['temperature'].values * units.degC
    Td = sounding['dewpoint'].values * units.degC
    u = sounding['u_wind'].values
    v = sounding['v_wind'].values
    date = sounding['time'][1]
    height = sounding['height'].values

    #-- Calculating variables
    lcl_pressure, lcl_temperature = mpcalc.lcl(p[0] * units.hPa, T[0], Td[0])
    prof = mpcalc.parcel_profile(p * units.hPa, T[0], Td[0]).to('degC')
    p_, u_, v_ = pressure_interval(p, u, v)
    cape, cin = mpcalc.cape_cin(p * units.hPa, T, Td, prof)
    try: cape = int(round(cape * units.kilogram/units.joule))
    except ValueError: cape = 0
    try: cin = int(round(cin * units.kilogram/units.joule))
    except ValueError: cin = 0
        
    #-- Start to plot
    fig = plt.figure(figsize=(7, 6))
    fig.set_facecolor('w')
    skew = SkewT(fig, rotation=45, subplot=gs[:, :2])
    #--- Grid
    gs = gridspec.GridSpec(3, 3)
    
    #--- Limits and title
    skew.ax.set_xlim(-30, 60)
    skew.ax.set_ylim(1000, 100)
    skew.ax.set_title('Station ' + station + '\n' + str(date) + ' UTC', weight='bold', stretch='condensed', size='x-large')
    skew.ax.set_xlabel('T, Td (deg. Celsius)', weight='bold', stretch='condensed', size='large')
    skew.ax.set_ylabel('p (hPa)', weight='bold', stretch='condensed', size='large')

    #--- Adding data
    skew.plot(p, T, 'r', linewidth=2)
    skew.plot(p, Td, 'g', linewidth=2)
    skew.plot(lcl_pressure, lcl_temperature, 'ko', markerfacecolor='black')
    skew.plot(p, prof, 'k', linewidth=1)
    if cin > 0: skew.shade_cin(p, T, prof)
    if cape > 0: skew.shade_cape(p, T, prof)
    skew.plot_dry_adiabats(linewidth=1)
    skew.plot_moist_adiabats(linewidth=1)
    skew.plot_mixing_lines(linewidth=1)
    skew.plot_barbs(p_, u_, v_, flip_barb=True)

    #--- Adding hodograph
    ax = fig.add_subplot(gs[0, -1])
    h = Hodograph(ax, component_range=60.)
    h.add_grid(increment=20)
    h.plot(u_, v_, linewidth=1.5, c='black')
        
    #--- (Trying to) add a colorbar for colored hodograph
    # gradient = np.linspace(0, 1, 256)
    # gradient = np.vstack((gradient, gradient))
    # ax = fig.add_axes([0.67, 0.6, 0.23, 0.01])
    # ax.imshow(gradient, aspect='auto', cmap=plt.get_cmap('viridis'))
    # ax.set_xlabel('Height (km)')
    # ax.yaxis.set_ticks([])
    # ax.xaxis.set_ticklabels(height[i]/256)

    #--- Adding CAPE, CIN values
    ax2 = fig.add_axes([0.67, 0.5, 0.19, 0.1])
    ax2.annotate('CAPE = ' + str(cape) + ' J/kg', xy=(0, 0.5), size='large')
    ax2.annotate('CIN = ' + str(cin) + ' J/kg', xy=(0, 0), size='large')
    ax2.set_axis_off()

    #-- Saving the figure
    plt.savefig(save_path + 'sounding_' + station + datetime.datetime.strftime(date, '%Y%m%d%H') + 'UTC.png', dpi=300, transparent=True, bbox_inches='tight')
    plt.close()
    
    return 'Sounding figure for ' + str(date) + ' done!'

## Defining path to save the figures, dates and station for the selected cases

In [5]:
save_path = "figures/"

cases = [datetime.datetime(2016, 12, 25, 12),   #-- Case 2016-12-25
         datetime.datetime(2017, 1, 31, 12),    #-- Case 2017-01-31
         datetime.datetime(2017, 3, 14, 12),    #-- Case 2017-03-14
         datetime.datetime(2017, 11, 15, 12),   #-- Case 2017-11-15
         datetime.datetime(2017, 11, 16, 0)]    #-- Case 2017-11-16 - problem on 12 UTC

station = 'SBMT'

## Acquiring data

In [6]:
soundings = [x for case in cases for x in acquire_sounding_data(case)]

## First look at the data

In [25]:
soundings[14][20:60]

,pressure,height,temperature,dewpoint,direction,speed,u_wind,v_wind,station,station_number,time,latitude,longitude,elevation
20,500.0,5870,-7.9,-32.9,45.0,7.0,-4.949747e+00,-4.949747e+00,SBMT,83779,2017-11-16,-23.52,-46.63,722.0
21,499.0,5886,-7.9,-32.9,46.0,7.0,-5.035379e+00,-4.862609e+00,SBMT,83779,2017-11-16,-23.52,-46.63,722.0
22,442.0,6813,-16.7,-31.7,125.0,8.0,-6.553216e+00,4.588611e+00,SBMT,83779,2017-11-16,-23.52,-46.63,722.0
23,404.0,7486,-19.1,-50.1,184.0,8.0,5.580518e-01,7.980512e+00,SBMT,83779,2017-11-16,-23.52,-46.63,722.0
24,400.0,7560,-19.5,-44.5,190.0,8.0,1.389185e+00,7.878462e+00,SBMT,83779,2017-11-16,-23.52,-46.63,722.0
25,397.0,7616,-19.7,-45.7,188.0,9.0,1.252558e+00,8.912413e+00,SBMT,83779,2017-11-16,-23.52,-46.63,722.0
26,393.0,7689,-20.3,-46.3,185.0,10.0,8.715574e-01,9.961947e+00,SBMT,83779,2017-11-16,-23.52,-46.63,722.0
27,361.0,8299,-25.0,-51.0,165.0,13.0,-3.364648e+00,1.255704e+01,SBMT,83779,2017-11-16,-23.52,-46.63,722.0
28,330.0,8945,-30.0,-56.0,190.0,15.0,2.604723e+00,1.477212e+01,SBMT,83779,2017-11-16,-23.52,-46.63,722.0
29,304.0,9535,-34.6,-60.6,195.0,17.0,4.399924e+00,1.642074e+01,SBMT,83779,2017-11-16,-23.52,-46.63,722.0


## Creating figures

In [8]:
[extract_vars_plot_save_figs(data, station, save_path) for data in soundings]

/home/camila/anaconda2/lib/python2.7/site-packages/metpy/calc/tools.py:161: RuntimeWarning: invalid value encountered in greater
  mask = sign_change > 0
/home/camila/anaconda2/lib/python2.7/site-packages/metpy/calc/thermo.py:1327: RuntimeWarning: invalid value encountered in greater
  keep_idx = np.ediff1d(x, to_end=[1]) > 0
/home/camila/anaconda2/lib/python2.7/site-packages/pint/quantity.py:1065: RuntimeWarning: invalid value encountered in less
  return op(self._convert_magnitude_not_inplace(UnitsContainer()), other)
/home/camila/anaconda2/lib/python2.7/site-packages/pint/quantity.py:1065: RuntimeWarning: invalid value encountered in greater
  return op(self._convert_magnitude_not_inplace(UnitsContainer()), other)
/home/camila/anaconda2/lib/python2.7/site-packages/pint/quantity.py:1070: RuntimeWarning: invalid value encountered in less
  return op(self._magnitude, other._magnitude)


['Sounding figure for 2017-01-31 12:00:00 done!',
 'Sounding figure for 2017-01-30 12:00:00 done!',
 'Sounding figure for 2017-01-29 12:00:00 done!',
 'Sounding figure for 2017-01-28 12:00:00 done!',
 'Sounding figure for 2017-03-14 12:00:00 done!',
 'Sounding figure for 2017-03-12 12:00:00 done!',
 'Sounding figure for 2017-03-11 12:00:00 done!',
 'Sounding figure for 2017-11-15 12:00:00 done!',
 'Sounding figure for 2017-11-15 00:00:00 done!',
 'Sounding figure for 2017-11-14 12:00:00 done!',
 'Sounding figure for 2017-11-14 00:00:00 done!',
 'Sounding figure for 2017-11-13 00:00:00 done!',
 'Sounding figure for 2017-11-12 12:00:00 done!',
 'Sounding figure for 2017-11-12 00:00:00 done!',
 'Sounding figure for 2017-11-16 00:00:00 done!',
 'Sounding figure for 2017-11-15 12:00:00 done!',
 'Sounding figure for 2017-11-15 00:00:00 done!',
 'Sounding figure for 2017-11-14 12:00:00 done!',
 'Sounding figure for 2017-11-14 00:00:00 done!',
 'Sounding figure for 2017-11-13 00:00:00 done!',
